## Задача (варіант 1)

Задача прийняття рішення полягає в оцінюванні чотирьох варіантів
деякого інноваційного товару за критерієм «перспективність попиту».
Результати парних порівнянь варіантів товару наступні: другий варіант
ненабагато кращий за перший і третій варіанти і суттєво кращий за
четвертий, перший варіант ненабагато кращий за третій і практично
такий самий як і четвертий варіант, третій варіант переважає четвертий
і ступінь переваги між рівною важливістю і несуттєвою перевагою.

## Матриця парних порівнянь

|Інтенсивність важливості | Якісна оцінка|
|-------------------------|--------------|
| 1 | однаково важливі |
| 3 | ненабагато важливіші |
| 5 | суттєво важливіші |
| 7 | значно важливіші |
| 9 | абсолютно важливіші |
2, 4, 6, 8 - проміжні

In [1]:
import numpy as np

Матриця парних порівнянь $D = \{d_{ij} | i, j = 1, \dots, 4\}$ альтернатив $A = \{a_i\}, i = 1, \dots, 4$.

Для мультиплікативних груп $d_{ij}$ вказує **у скільки** разів альтернатива $a_i$ переважає альтернативу $a_j$ вдносно критерію, за яким будується матриця $D$.

In [5]:
D = np.array([[1.0, 1/3.0, 3.0, 1.0],
              [3.0, 1.0, 3.0, 5.0],
              [1/3.0, 1/3.0, 1.0, 2.0],
              [1.0, 1/5.0, 1/2.0, 1.0]])
D

array([[1.        , 0.33333333, 3.        , 1.        ],
       [3.        , 1.        , 3.        , 5.        ],
       [0.33333333, 0.33333333, 1.        , 2.        ],
       [1.        , 0.2       , 0.5       , 1.        ]])

Перевірка матриці парних порівнянь на коректність(якщо альтернатива $a_i$ краща за альтернативу $a_j$ у $k$ разів, тоді $d_{ij} = k, d_{ji} = \frac{1}{k}$).

In [6]:
# check matrix on inverse symmetry property
def check_inverse_symmetry(matrix, group_type='multiplicative'):
    """
    Checks inverse synnetry property dependyng on type of the group - 'multiplicative' or 'additive'.
    """
    if group_type == 'multiplicative':
        f = lambda x: 1 / x
    elif group_type == 'additive':
        f = lambda x: -x
    else:
        raise ValueError('Wrong group type argument')
    
    for i in range(len(matrix)):
        for j in range(i, len(matrix[i])):
            if matrix[j][i] != f(matrix[i][j]):
                return False
    
    if group_type == 'multiplicative':
        for i in range(len(matrix)):
            for j in range(len(matrix[i])):
                if matrix[i][j] <= 0:
                    return false
    
    return True

print("Is matrix satisfies inverse symmetry property: {}".format(check_inverse_symmetry(D)))

Is matrix satisfies inverse symmetry property: True


## Ваги альтернатив

Метою спеціаліста з методу аналізу ієрархій є побудова ваг для кожної з альтернатив: $a_i, i = 1, \dots, 4$. Для цього існують різні методи. Нижче застосовано метод головного власного вектору(eigenvector method, EM)

### Eigenvector method (EM)

Шукаємо власні числа та відповідні їм власні вектори матриці парних порівнянь $D$

In [8]:
from numpy import linalg
# calculate eigenvalues 'w' and eigenvectors 'v' using NumPy package
w, v = linalg.eig(D)
for i in range(len(w)):
    print('Eigenvalue: {} \nCorresponding eigenvector: {}'.format(w[i], v[i]))
    print('*' * 40)

Eigenvalue: (4.309521917464972+0j) 
Corresponding eigenvector: [0.37381158+0.j 0.6885513 +0.j 0.6885513 -0.j 0.0444672 +0.j]
****************************************
Eigenvalue: (-0.17389889540919656+1.1475900512031707j) 
Corresponding eigenvector: [ 0.87037106+0.j         -0.4527623 -0.20393508j -0.4527623 +0.20393508j
 -0.98882619+0.j        ]
****************************************
Eigenvalue: (-0.17389889540919656-1.1475900512031707j) 
Corresponding eigenvector: [ 0.24799935+0.j         -0.16556011+0.38091261j -0.16556011-0.38091261j
  0.05138678+0.j        ]
****************************************
Eigenvalue: (0.038275873353423466+0j) 
Corresponding eigenvector: [ 0.20301587+0.j         -0.16068851-0.28458483j -0.16068851+0.28458483j
  0.13268321+0.j        ]
****************************************


Власний вектор, що відповідає максимальному власному числу буде шуканим набором ваг. З теорії відомо, що максимальне власне число буде дійсним, йому відповідатиме вектор власний вектор, що складатиметься з ваг одного знаку(всі від'ємні або всі додатні). Якщо отримуємо від'ємний вектор - множимо на $-1$. Отриманий вектор і є шуканим вектором ваг.

In [11]:
# eigenvector corresponding to max eigenvector
w_max = np.real(np.max(w))
print("Max eigenvalue: {}".format(w_max))
v_max = np.real(v[np.argmax(w)])
print("Corresponding eigenvector: {}".format(v_max))

Max eigenvalue: 4.309521917464972
Corresponding eigenvector: [0.37381158 0.6885513  0.6885513  0.0444672 ]


### CR

Табличні значення середнього значення індексів узгодженності для заповнених випадковим чином матриць парних порівнянь

In [12]:
# dependence of MRCI from dimension 'n' of Pair Comparison Matrix
mrci = np.array([0, 0, 0.52, 0.89, 1.11, 1.25, 1.35, 1.4, 1.45, 1.49, 1.52, 1.54, 1.56, 1.58, 1.59])

In [15]:
n = D.shape[0]
print(n)

4


Consistency index computing:

In [17]:
CI = (w_max - n) / (n - 1)
print(CI)

0.1031739724883239


Compute consistency ratio:

In [18]:
CR = CI / mrci[n - 1]
print(CR)

0.1159258117846336


**МПП недопустимо неузгоджена, бо СІ перевищує попрогове значення для n = 4: 0.08**

## Оцінка узгодженності МПП

### За визначенням
CR < Threshold, тому допустимо узгоджена - може використовуватися для обрахунку ваг.

In [21]:
counter = 0
for i in range(D.shape[0]):
    for j in range(i+1, D.shape[0]):
        for k in range(j+1, D.shape[0]):
            counter += 1
            if D[i][j] != D[i][k] * D[k][j]:
                print("Error: i == {}, j == {}, k == {}".format(i, j, k))
                print("{} != {}".format(D[i][j], D[i][k] * D[k][j]))

Error: i == 0, j == 1, k == 2
0.3333333333333333 != 1.0
Error: i == 0, j == 1, k == 3
0.3333333333333333 != 0.2
Error: i == 0, j == 2, k == 3
3.0 != 0.5
Error: i == 1, j == 2, k == 3
3.0 != 2.5


# Фінальні ваги

In [22]:
weights = v_max / np.linalg.norm(v_max, 2)
print("Final weights: {}".format(weights))

Final weights: [0.35805998 0.65953728 0.65953728 0.04259345]


Перевірка

In [23]:
print(np.sum(np.power(weights, 2)))

1.0000000000000002
